In [1]:
import tifffile as t
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import zarr
from matplotlib.widgets import Slider
import dask.array as da
import dask
import timeit
import matplotlib

matplotlib.use("QtAgg")

In [2]:
def imshow(volume):
    # Initialize plot.
    fig, ax = plt.subplots()
    plt.subplots_adjust(left=0.25, bottom=0.25)

    # Assuming volume is a zarr array and we're slicing along the first axis.
    slice_idx = volume.shape[0] // 2  # Start in the middle of the volume
    img = ax.imshow(volume[slice_idx, :, :], cmap="gray")

    # Position of the slider
    ax_slider = plt.axes([0.25, 0.1, 0.65, 0.03])

    # Slider properties
    slider = Slider(
        ax=ax_slider,
        label="Slice",
        valmin=0,
        valmax=volume.shape[0] - 1,
        valinit=slice_idx,
        valstep=1,
    )

    # Update function for the slider
    def update(val):
        slice_idx = int(slider.val)
        img.set_data(volume[slice_idx, :, :])
        fig.canvas.draw_idle()

    # Call update function on slider value change
    slider.on_changed(update)

    # Function to handle keyboard input (left and right arrow keys)
    def on_key(event):
        if event.key == "left":
            new_val = max(slider.val - 1, slider.valmin)  # Decrease slider value
        elif event.key == "right":
            new_val = min(slider.val + 1, slider.valmax)  # Increase slider value
        else:
            return
        slider.set_val(new_val)  # Update the slider value

    # Connect the keyboard event to the on_key function
    fig.canvas.mpl_connect("key_press_event", on_key)

    plt.show()


def interactive_slice_viewer(volume, axis=0):
    # Crop the volume
    # Retain all slices and height, take 1/100th of the width
    cropped_volume = volume[:, :, : volume.shape[2] // 100]

    # Initialize plot.
    fig, ax = plt.subplots()
    plt.subplots_adjust(left=0.25, bottom=0.25)

    # Set initial slice index (middle of the volume along the specified axis)
    slice_idx = cropped_volume.shape[axis] // 2

    # Function to get the slice based on the axis
    def get_slice(idx):
        if axis == 0:
            return cropped_volume[idx, :, :]
        elif axis == 1:
            return cropped_volume[:, idx, :]
        elif axis == 2:
            return cropped_volume[:, :, idx]
        else:
            raise ValueError("Invalid axis. Axis must be 0, 1, or 2.")

    # Display initial slice
    img = ax.imshow(get_slice(slice_idx), cmap="gray", origin="lower")

    # Create slider for navigating slices
    ax_slider = plt.axes([0.25, 0.1, 0.65, 0.03])
    slider = Slider(
        ax=ax_slider,
        label="Slice",
        valmin=0,
        valmax=cropped_volume.shape[axis] - 1,
        valinit=slice_idx,
        valstep=1,
    )

    # Update function for the slider
    def update(val):
        idx = int(slider.val)
        img.set_data(get_slice(idx))
        fig.canvas.draw_idle()

    slider.on_changed(update)

    # Keyboard controls for slider
    def on_key(event):
        if event.key == "left":
            new_val = max(slider.val - 1, slider.valmin)
        elif event.key == "right":
            new_val = min(slider.val + 1, slider.valmax)
        else:
            return
        slider.set_val(new_val)

    fig.canvas.mpl_connect("key_press_event", on_key)

    plt.show()


def interactive_slice_viewer_dask(dask_volume, axis=0):
    # Initialize plot.
    fig, ax = plt.subplots()
    plt.subplots_adjust(left=0.25, bottom=0.25)

    # Set initial slice index (middle of the volume along the specified axis)
    slice_idx = dask_volume.shape[axis] // 2

    # Function to get the slice based on the axis
    def get_slice(idx):
        if axis == 0:
            return dask_volume[idx, :, :].compute()  # Compute the slice
        elif axis == 1:
            return dask_volume[:, idx, :].compute()  # Compute the slice
        elif axis == 2:
            return dask_volume[:, :, idx].compute()  # Compute the slice
        else:
            raise ValueError("Invalid axis. Axis must be 0, 1, or 2.")

    # Display initial slice
    img = ax.imshow(get_slice(slice_idx), cmap="gray", origin="lower")

    # Create slider for navigating slices
    ax_slider = plt.axes([0.25, 0.1, 0.65, 0.03])
    slider = Slider(
        ax=ax_slider,
        label="Slice",
        valmin=0,
        valmax=dask_volume.shape[axis] - 1,
        valinit=slice_idx,
        valstep=1,
    )

    # Update function for the slider
    def update(val):
        idx = int(slider.val)
        img.set_data(get_slice(idx))
        fig.canvas.draw_idle()

    slider.on_changed(update)

    # Keyboard controls for slider
    def on_key(event):
        if event.key == "left":
            new_val = max(slider.val - 1, slider.valmin)
        elif event.key == "right":
            new_val = min(slider.val + 1, slider.valmax)
        else:
            return
        slider.set_val(new_val)

    fig.canvas.mpl_connect("key_press_event", on_key)

    plt.show()

def create_figure():
    plt.ion()  # Turn on interactive mode
    fig, ax = plt.subplots()
    def on_close(event):
        plt.ioff()  # Turn off interactive mode
        plt.close(event.canvas.figure)  # Close the figure
    fig.canvas.mpl_connect('close_event', on_close)
    return fig, ax

In [3]:
seq = t.TiffSequence(
    "Ti-5Fe_05_Ti64_350W_700mmpmin_2p4gpmin_layer-01_S0001_S0001/*.tif",
)


vol = zarr.open(seq.aszarr())
partial_vol = np.load("partial_vol.npy")
right_side = np.transpose(partial_vol, (2, 1, 0))

In [4]:
fig, ax = create_figure()
ax.imshow(right_side[0], cmap="gray", origin="lower")
plt.show()


In [5]:
# Select a slice from 'right_side' volume
slice_idx = 50  # Change as needed
original_slice = right_side[slice_idx]

# Resize only the width (x-axis) of the slice for visualization

resized_slice = original_slice[:, ::100]

In [7]:
fig, ax = create_figure()
ax.imshow(resized_slice, cmap='gray')
line_coords_resized = []

def on_click(event):
    if fig.canvas.toolbar.mode == '':
        # Store the x and y coordinates of the click event in the resized space
        line_coords_resized.append((event.xdata, event.ydata))
        if len(line_coords_resized) == 2:
            ax.plot([line_coords_resized[0][0], line_coords_resized[1][0]], [line_coords_resized[0][1], line_coords_resized[1][1]], 'r-')
            fig.canvas.draw()
            fig.canvas.mpl_disconnect(cid)

cid = fig.canvas.mpl_connect('button_press_event', on_click)
plt.show()

In [8]:
if len(line_coords_resized) == 2:
    # Map only the x-coordinates back to the original image space
    line_coords_original = [
        (0, line_coords_resized[0][1]),
        (original_slice.shape[1], line_coords_resized[1][1])
    ]
    print(f"Line in resized image space: {line_coords_resized}")
    print(f"Line in original image space: {line_coords_original}")


Line in resized image space: [(-0.2903711854544859, 440.01128091992433), (894.2203414507134, 500.448211950278)]
Line in original image space: [(0, 440.01128091992433), (89470, 500.448211950278)]


In [9]:
import matplotlib.pyplot as plt

# Plot the original slice
fig, ax = create_figure()
ax.imshow(original_slice, cmap='gray')  # Display the original slice

# Draw the line on the original slice
# Extract x and y coordinates for the start and end points of the line
x_coords = [line_coords_original[0][0], line_coords_original[1][0]]
y_coords = [line_coords_original[0][1], line_coords_original[1][1]]

# Plot the line on the original image
ax.plot(x_coords, y_coords, 'r-')  # 'r-' means red line

# Display the plot with the line
plt.title("Original Slice with Converted Line")
plt.show()


"""
We can calculate each slice shift upwards or downwards by calculating the chord at the radius (slice).
To know the direction of the shift, we have to check if the line given by the user is above or below the horizontal line.
The horizontal line must be drawn from the first point (0, py) to (width, py) where py is the y-coordinate of the first point.
Then get the angle in absolute value between the horizontal line and the line given by the user and calculate each cord.
Check if this can be done like that, we must probably have to change behaviour to process one slice at a time.
Dask is becoming more neccessary with this approach, it may be useful to instead of save to disk perform the operations
concurrently with dask and assign back to the zarr array, reading it in write mode.
"""

'\nWe can calculate each slice shift upwards or downwards by calculating the chord at the radius (slice).\nTo know the direction of the shift, we have to check if the line given by the user is above or below the horizontal line.\nThe horizontal line must be drawn from the first point (0, py) to (width, py) where py is the y-coordinate of the first point.\nThen get the angle in absolute value between the horizontal line and the line given by the user and calculate each cord.\nCheck if this can be done like that, we must probably have to change behaviour to process one slice at a time.\nDask is becoming more neccessary with this approach, it may be useful to instead of save to disk perform the operations\nconcurrently with dask and assign back to the zarr array, reading it in write mode.\n'

In [12]:
dx = line_coords_original[1][0] - line_coords_original[0][0]
dy = line_coords_original[1][1] - line_coords_original[0][1]

# Calculate the angle in radians
angle_radians = np.arctan2(dy, dx)

# Convert the angle to degrees
angle_degrees = np.degrees(angle_radians)

# Determine the direction of the line
direction = "upwards" if dy < 0 else "downwards" if dy > 0 else "horizontal"


print(f"Angle (in radians): {angle_radians}")
print(f"Angle (in degrees): {angle_degrees}")
print(f"Direction of the line: {direction}")
print(f"{line_coords_original[0]} -> {line_coords_original[1]}")
# Same as above but with resized coordinates
print(f"{line_coords_resized[0]} -> {line_coords_resized[1]}")

Angle (in radians): 0.0006754992940416939
Angle (in degrees): 0.03870325861265565
Direction of the line: downwards
(0, 440.01128091992433) -> (89470, 500.448211950278)
(-0.2903711854544859, 440.01128091992433) -> (894.2203414507134, 500.448211950278)


In [38]:
import math
from concurrent.futures import ThreadPoolExecutor

iterator = range(vol.shape[0]) if direction == "downwards" else reversed(range(vol.shape[0]))
for i in iterator:
    # Slice the volume
    _slice = vol[i, :, :]
    shift_size = math.ceil(2 * i * math.sin(angle_radians / 2))
    print(f"Shifting slice {i} by {shift_size} pixels")
    shifted_slice = np.roll(_slice, -shift_size, axis=0)
    t.imwrite(f"./shifted/shifted_slice_{i}.tif", shifted_slice)


Shifting slice 0 by 0 pixels
Shifting slice 1 by 1 pixels
Shifting slice 2 by 1 pixels
Shifting slice 3 by 1 pixels
Shifting slice 4 by 1 pixels
Shifting slice 5 by 1 pixels
Shifting slice 6 by 1 pixels
Shifting slice 7 by 1 pixels
Shifting slice 8 by 1 pixels
Shifting slice 9 by 1 pixels
Shifting slice 10 by 1 pixels
Shifting slice 11 by 1 pixels
Shifting slice 12 by 1 pixels
Shifting slice 13 by 1 pixels
Shifting slice 14 by 1 pixels
Shifting slice 15 by 1 pixels
Shifting slice 16 by 1 pixels
Shifting slice 17 by 1 pixels
Shifting slice 18 by 1 pixels
Shifting slice 19 by 1 pixels
Shifting slice 20 by 1 pixels
Shifting slice 21 by 1 pixels
Shifting slice 22 by 1 pixels
Shifting slice 23 by 1 pixels
Shifting slice 24 by 1 pixels
Shifting slice 25 by 1 pixels
Shifting slice 26 by 1 pixels
Shifting slice 27 by 1 pixels
Shifting slice 28 by 1 pixels
Shifting slice 29 by 1 pixels
Shifting slice 30 by 1 pixels
Shifting slice 31 by 1 pixels
Shifting slice 32 by 1 pixels
Shifting slice 33 by

In [13]:
import math
import os
import shutil
import time
import tifffile as t
import numpy as np
from concurrent.futures import ThreadPoolExecutor
import psutil

cpu_logical_count = psutil.cpu_count(logical=True)
cpu_physical_count = psutil.cpu_count(logical=False)

print(f"Logical CPUs (threads): {cpu_logical_count}")
print(f"Physical CPUs (cores): {cpu_physical_count}")

cpu_count = os.cpu_count()
print(f"Number of CPU cores: {cpu_count}")

# Define the function to process a single slice (for concurrent processing)
def process_slice(i):
    _slice = vol[i, :, :]
    shift_size = math.ceil(2 * i * math.sin(angle_radians / 2))
    # print(f"Shifting slice {i} by {shift_size} pixels (concurrent)")
    shifted_slice = np.roll(_slice, -shift_size, axis=0)
    t.imwrite(f"./shifted_concurrent/shifted_slice_{i}.tif", shifted_slice)

# Create the 'shifted' and 'shifted_concurrent' folders if they don't exist
shifted_folder = "./shifted"
shifted_concurrent_folder = "./shifted_concurrent"
os.makedirs(shifted_folder, exist_ok=True)
os.makedirs(shifted_concurrent_folder, exist_ok=True)

# # Time the sequential processing
# start_time_seq = time.time()
iterator = range(vol.shape[0]) if direction == "downwards" else reversed(range(vol.shape[0]))
# for i in iterator:
#     _slice = vol[i, :, :]
#     shift_size = math.ceil(2 * i * math.sin(angle_radians / 2))
#     # print(f"Shifting slice {i} by {shift_size} pixels (sequential)")
#     shifted_slice = np.roll(_slice, -shift_size, axis=0)
#     t.imwrite(f"{shifted_folder}/shifted_slice_{i}.tif", shifted_slice)
# end_time_seq = time.time()
# print(f"Sequential processing time: {end_time_seq - start_time_seq} seconds.")

# Remove the 'shifted' folder
shutil.rmtree(shifted_folder)

# Time the concurrent processing
start_time_concurrent = time.time()
with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:  # Adjust max_workers as needed
    executor.map(process_slice, iterator)
end_time_concurrent = time.time()
print(f"Concurrent processing time: {end_time_concurrent - start_time_concurrent} seconds.")


Logical CPUs (threads): 20
Physical CPUs (cores): 12
Number of CPU cores: 20


Sequential processing time: 686.0818467140198 seconds.
Concurrent processing time: 614.9080040454865 seconds.
